In [2]:
from PIL import Image 
import glob
import numpy as np
import random

In [3]:
#making list to store img data and clearing them to not have residue
img_list=[]
img1_list=[]
img_list.clear()
img1_list.clear()

#making arrays to store notes and memes    
xm=np.empty((256,256,3))
xn=np.empty((256,256,3))

#importing memes images
for file in glob.glob('C:\\Users\\Gourav\\Desktop\\programs\\DL\\reshaped\\Memes\\*.jpg'):
    img = Image.open(file) 
    img_list.append(img)

In [4]:
#storing memes data as arrays
xm=np.array(img_list[0])
xm=xm.reshape((256*256*3,1))

for i in range(len(img_list)-1):
    temp=np.array(img_list[i+1])
    temp=temp.reshape((256*256*3,1))
    xm=np.append(xm,temp,axis=1)

In [5]:
#importing notes images
for file1 in glob.glob('C:\\Users\\Gourav\\Desktop\\programs\\DL\\reshaped\\Notes\\*.jpg'):
    img1 = Image.open(file1) 
    img1_list.append(img1)

#storing notes data as arrays
xn=np.array(img1_list[0])
xn=xn.reshape((256*256*3,1))
    
for i in range(len(img1_list)-1):
    temp1=np.array(img1_list[i+1])
    temp1=temp1.reshape((256*256*3,1))
    xn=np.append(xn,temp1,axis=1)

In [6]:
#making input vector by appending memes and notes array
x=np.append(xm,xn,axis=1)

In [7]:
#making labels for input
#0 means memes 1 means notes
y0=np.zeros((1,800))
y1=np.ones((1,800))
y=np.append(y0,y1)
Y=y.reshape(1,1600)
print(y.shape)

(1600,)


In [8]:
#shuffle data
permutation = list(np.random.permutation(1600))
x_arr=np.copy(x)
y_arr=np.copy(Y)

random.seed(1)
np.random.shuffle(np.transpose(x_arr))
np.transpose(x_arr)

random.seed(1)
shuff_y=np.random.shuffle(np.transpose(y_arr))
np.transpose(y_arr)

print(y_arr.shape)
print(x_arr.shape)

(1, 1600)
(196608, 1600)


In [9]:
#make test train validation
x_train=np.array(x_arr[:1500])
y_train=np.array(y_arr[:1500])

x_test=np.array(x_arr[1500:1550])
y_test=np.array(y_arr[1500:1550])

x_val=np.array(x_arr[1550:1600])
y_val=np.array(y_arr[1550:1600])

In [16]:
#make one hot
def one_hot(x):
    Y = np.zeros((2,x.shape[0]))
    for n,i in enumerate(range(len(x))):
        Y[int(x[0][i]),i]=1
    return Y

Y_train=np.array(one_hot(y_train))
Y_test=np.array(one_hot(y_test))
Y_val=np.array(one_hot(y_val))

In [17]:
#making var to store weights
def initialize_parameters(dims):
    parameters={}
    l = len(dims)
    for i in range(1,l):
        parameters['W'+str(i)] = np.random.randn(dims[i],dims[i-1])*0.01
        parameters['b'+str(i)] = np.zeros((dims[i],1))
    return parameters

In [22]:
#activation functions
def sigmoid(Z):
    S = 1 / (1 + np.exp(-Z))
    return S

def sigmoid_back(dA, Z):
  A = 1/(1+np.exp(-Z))
  return dA * (1 - A) * A

def relu(Z):
    R = np.maximum(0, Z)
    return R

def Relu_back(z):
    z[z<=0] = 0
    z[z>0] = 1
    return z

def tanh(z):
    return (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [39]:
#forward propogation model
def forward_propagation(X, parameters):
    cache = {}
    l = len(parameters)//2
    cache['A0'] = X
    for i in range(1,l+1):
        cache['Z'+str(i)] = np.dot(parameters['W'+str(i)],cache['A'+str(i-1)]) + parameters['b'+str(i)]
        
        cache['A'+str(i)] = relu(cache['Z'+str(i)])
        
    cache['A'+str(l)] = softmax(cache['Z'+str(l)])
            
    return cache

In [20]:
#cost function 
def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = -1 / m * np.sum(Y * np.log(AL) + (1-Y) * np.log(1-AL))
    return cost

In [23]:
#back prop model
def back_propagation(X, Y, parameters, cache):
    m = Y.shape[1]
    l = len(parameters)//2
    back_cache = {}
    
    
    dZ = cache['A'+str(l)] - Y
    back_cache['dW'+str(l)] = (1/m)*np.dot(dZ, cache['A'+str(l-1)].T)
    back_cache['db'+str(l)] = (1/m)*np.sum(dZ, axis=1, keepdims = True)
    
    for i in reversed(range(1,l)):
        dZ = np.dot(parameters['W'+str(i+1)].T,dZ)*Relu_back(cache['Z'+str(i)])
        back_cache['dW'+str(i)] = (1/m)*np.dot(dZ, cache['A'+str(i-1)].T)
        back_cache['db'+str(i)] = (1/m)*np.sum(dZ, axis=1, keepdims=True)
     
    return back_cache

In [24]:
#updates weights and biases
def update(paramters, back_cache, lr):
    l = len(parameters)//2
    for i in range(1,l+1):
        parameters['W'+str(i)] -= lr*back_cache['dW'+str(i)]
        parameters['b'+str(i)] -= lr*back_cache['db'+str(i)]
        
    return parameters

In [25]:
#prediction
def predict(pred, Y):
  temp=[]
  pred = pred.T
  for i in pred:
    temp.append(np.argmax(i))
  
  count = 0

  for i in range(len(temp)):
    if temp[i] == Y[i]:
      count+=1
      
  return count

In [34]:
parameters = initialize_parameters([196608, 5000,100, 2])

In [35]:
x_train = x_train/255
x_val = x_val/255
x_test = x_test/255


In [40]:
costs=[]
for i in range(100):
    cache = forward_propagation(x_train, parameters)
    c = compute_cost(cache['A'+str(len([196608, 5000,100, 2])-1)],Y_train)
    if i % 10 == 0:
            print("Cost after iteration %i: %f" %(i, c))
    costs.append(c)
    grads = back_propagation(x_train, Y_train, parameters, cache)
    
    parameters = update(parameters, grads, 0.008)

ValueError: shapes (5000,196608) and (1500,1600) not aligned: 196608 (dim 1) != 1500 (dim 0)